In [1]:
from data_base import *

data = DataBase(summary=True, max_size=10**5)

Whole text size: 36161157 so used 100000
Alphabet length: 160
Alphabet:  !"#%&'()*,-./0123456789:;<=>?@[]^_`abcdefghijklmnopqrstuvwxyz{|}~§«°·»½¾ßàáâäçèéêëíîïòóôöùúûüýœ̀́γεθιλοστυώабвгдежзийклмнопрстуфхцчшщъыьэюяёєіїљќѣ‑–—’“”„…№


In [3]:
import numpy as np

input_len = 30

n = 100_000
data.text_size = n - input_len
print('Train data size:', data.text_size)

Xf = np.zeros((data.text_size, input_len), dtype=np.float64)
Yf = np.zeros((data.text_size,), dtype=np.float64)


def text_to_indexes(text):
    return np.array([data.letter_index[s] for s in text])


for i in tqdm(range(input_len, n)):
    fragment = data.whole_text[i-input_len : i]
    Xf[i-input_len] = text_to_indexes(fragment)
    
    Yf[i-input_len] = data.letter_index[data.whole_text[i]]

Train data size: 99970


100%|██████████| 99970/99970 [00:01<00:00, 75959.09it/s] 


In [4]:
from scipy.fft import fft, ifft

convergence = 0

for i in tqdm(range(data.text_size)):
	a = ifft(Xf[i])

	pred = np.zeros(a.shape[0]*2, dtype=np.complex_)
	pred[::2] = a

	pred = np.mean(np.real(fft(pred))[15:].round())

	convergence += abs(Yf[i] - pred)

convergence /= n
convergence /= data.alph_len

print('Non-convergence with fourier:', convergence)

100%|██████████| 99970/99970 [00:04<00:00, 21988.13it/s]

Non-convergence with fourier: 0.22318000833333218


In [5]:
test_text = prepare_text("Если посмотреть из чего получается оригинальная мысль, то можем выделить следующие факторы: личные ценности, культурный опыт, перспектива и взгляд на мир, эмоциональные и интуитивные компоненты. Из этого и складывается индивидуальность творца-человека. ИИ не обладает эмоциями. Но что из этого не может быть имитировано? Он не может обладать ничем из перечисленного, ведь это все присуще живым и мыслящим существам, а ИИ не живой; однако он может все это впитать и воссоздать в себе. Эти факторы - следствие всего пережитого, всего накопленного и отпечатанного опыта. А впитывать опыт - то, что ИИ и делают и способны делать это лучше нас и в гораздо большем объеме, чем способны люди, ведь ИИ не органичен физиологическими факторами.")

r = test_text[:input_len]

for i in tqdm(range(input_len, len(test_text))):
	#frag = test_text[i-input_len : i]
	fragment = r[-input_len:]
	fragment = text_to_indexes(fragment)
	a = ifft(fragment)
	pred = np.zeros(a.shape[0]*2, dtype=np.complex_)
	pred[::2] = a

	prediction = round(np.mean(np.real(fft(pred))[15:].round()))

	#print(prediction, data.letter_index[test_text[i]])

	if 1 - abs(data.letter_index[test_text[i]] - prediction) / data.alph_len > 0.9 and test_text[i] != ' ':
	#if letter_of_pred(prediction) == test_text[i] and test_text[i] != ' ':
		r += ''
	else:
		r += test_text[i]

print(r)
print('Original length:', len(test_text))
print('Compressed length:', len(r))
print('Compression:', len(r) / len(test_text))

100%|██████████| 704/704 [00:00<00:00, 12835.26it/s]

если посмотреть из чего получатся орья ысь, то можм выелить слующ фкторы: личны цнност, культурны опыт, прспкт и взгля н мир, эмоциональны и интутны компонты.  этоо и складыется инульность трц-чо.  не обладает эмоциями. но что из этого не может быть имитироно? он не может облть ничм из перечислнноо, еь это все присущ жиым и мыслящим сущст,  ии не живой; однако он может все это впитть и воссозть  себе. эти факторы - следствие всего пржитоо, всего нкоплнноо  отпечтнноо опыт.  впитыть опыт - то, что ии и делают и способны делть это лучш нс и в гораздо большем оъм, чем спосоны люи, ведь ии не органичен физиолоичс фктор.
Original length: 734
Compressed length: 623
Compression: 0.8487738419618529
